In [2]:
import time
import numpy as np
from Conways import*

pygame 2.0.2 (SDL 2.0.16, Python 3.9.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
env = conways(size = 10, max_it=1000)
print(env.run())

5


In [4]:
x = np.zeros(shape=(10, 10), dtype=bool)
x[1, 1] = True
x[1, 2] = True
x[2, 1] = True
x[2, 2] = True


In [5]:
env.reset(x, offset=False)
actions = env.run()
print(actions)

inf


In [12]:
alpha = 0.1 
max_iterations = 100000
gamma = 0.995

total = 0
tot_reward = 0
prev_reward = 0

q_table_row_limit = 1000

epsilon = 1
epsilon_decay = 0.99995

In [69]:
q_table = np.random.uniform(low=0, high=1, size=[q_table_row_limit, 10, 10])
q_table.shape

print(np.argmax(q_table[0]))
print(np.max(q_table[0]))
print(q_table[0])

37
0.9868778767122895
[[0.43597294 0.17075194 0.70323902 0.79364689 0.7212616  0.23870057
  0.33351781 0.34004142 0.09620045 0.46153998]
 [0.9253357  0.00443245 0.54500834 0.72538512 0.79862045 0.40248761
  0.50839658 0.42021774 0.38044416 0.55525096]
 [0.26858018 0.1539474  0.3515666  0.88556765 0.12826753 0.84943665
  0.90122382 0.89085294 0.66251069 0.19253059]
 [0.78915762 0.32820731 0.14567329 0.95737662 0.08181847 0.13123711
  0.75432058 0.98687788 0.67441357 0.79528384]
 [0.48757673 0.54501297 0.36673892 0.85695461 0.80322971 0.68193952
  0.69987424 0.73457059 0.8708963  0.45479316]
 [0.87210277 0.26379741 0.79462286 0.73906619 0.70304357 0.43066846
  0.2200744  0.00911005 0.2638947  0.26069749]
 [0.27653386 0.38078055 0.97185337 0.41718749 0.31915725 0.1726408
  0.09992213 0.72486477 0.55031117 0.65733778]
 [0.19837549 0.22784423 0.42290038 0.88781244 0.38763772 0.17937834
  0.21776417 0.61195704 0.41276754 0.76858205]
 [0.68227035 0.45054126 0.71948747 0.9473031  0.64422897 0.

In [71]:
state_table = {}
count = 0
def get_discrete_state(s):
    global count
    global state_table
    key = s
    if (key in state_table.keys()):
        return state_table[key]
    
    state_table[key] = count
    count += 1
    return count
    
def table_clear():
    global count
    global state_table

    state_table.clear()
    count = 0

In [70]:
current_state = np.zeros(shape=(10, 10), dtype=bool)
state_score = np.array(float('-inf') for i in range(q_table_row_limit))
#take initial action
index = np.argmax(q_table[discrete_state])
action = (index/10, index%10)
current_state[action[0], action[1]] = True
best_state = current_state
discrete_state = get_discrete_state(current_state)
env.reset(current_state)
state_score[discrete_state] = env.run()
best_reward = np.copy(current_state)

for i in range(max_iterations):
    t0 = time.time()
    

    if (i % 2000 == 0):
        print(f"Episode: {i}")

    
    #test possible actions after taking that action
    for j in range(q_table_row_limit-1):
        if np.random.random() > epsilon:
            index = np.argmax(q_table[discrete_state])
            action = (index/10, index%10)
        else:
            action = (np.random.randint(0, 10), np.random.randint(0, 10))

        current_state[action[0], action[1]] = True
        env.reset(current_state)

        new_discrete_state = get_discrete_state(current_state)
        state_score[discrete_state] = env.run()
        if (state_score[discrete_state] > best_reward):
            best_state = np.copy(current_state)

        max_future_q = np.max(q_table[new_discrete_state])
        current_q = q_table[(discrete_state, action[0], action[1])]

        new_q = (1 - alpha) * current_q + alpha * (state_score[discrete_state] + gamma * max_future_q)
        q_table[(discrete_state, action[0], action[1])] = new_q

        discrete_state = new_discrete_state
    
    reset q table and state indexing table
    current_state = np.copy(best_state)
    reward = np.max(state_score)
    if epsilon > 0.05: 
        if reward > prev_reward and i > 10000:
            epsilon = epsilon_decay ** (i - 10000)

            if i % 500 == 0:
                print("Epsilon: " + str(epsilon))
    
    t1 = time.time()
    iteration_time = t1-t0
    total += iteration_time

    tot_reward += reward
    prev_reward = reward

    if i % 1000 == 0: 
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = tot_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        tot_reward = 0


Episode: 0


NameError: name 'discrete_state' is not defined